In [1]:
#!/usr/bin/env python
# coding: utf-8

# import matplotlib.pyplot as plt
import os, sys
import geopandas as gpd
import datetime
import matplotlib.pyplot as plt
from pw_data import SIC25k


CRS = 'epsg:3413'
NRT_DAILY_ID = 'nsidcG10016v2nh1day'
VAR_NAME = 'cdr_seaice_conc'
GRID_CELL_AREA_ID = 'pstere_gridcell_N25k'

REGIONS = dict([('AlaskanArctic', 'arctic_sf.shp')])#,('NorthernBering', 'nbering_sf.shp'), ('EasternBering', 'ebering_sf.shp')])

# Instatiate SIC25k object and load grid area data
sic_m = SIC25k(NRT_DAILY_ID, VAR_NAME, CRS)
sic_m.load_area(GRID_CELL_AREA_ID)  

# Set this and previous years
thisyear = datetime.date.today().year
lastyear = thisyear - 1   

# Loop over each region and its corresponding shapefile
for name, shp in REGIONS.items():
    # List to store annual sea ice extent for each region and year
    extents = []
    # Load the shapefile for the region and transform it to the dataset's CRS
    alaska_shp = gpd.read_file(f'resources/akmarineeco/{shp}')  # Read shapefile
    alaska_shp_proj = alaska_shp.to_crs(CRS)  # Reproject the shapefile to match the dataset CRS

    # Loop over each year from 1995 to 2010, computing sea ice extent for each September 1 to August 31 period
    for year in range(lastyear, thisyear):
        # Subset the dataset by time (September 1 to August 31) and region (clip to the shapefile)
        ds, area = sic_m.subset_dim([f'{lastyear-1}-09-01', f'{lastyear}-08-31'], alaska_shp_proj)

        # Format the sea ice concentration data to binary (0 or 1) using a threshold of 0.15
        sic = sic_m.format_sic(ds, 0.15)  # Sea ice concentration thresholding

        # Compute the sea ice extent in square kilometers for the subset data
        ext = sic_m.compute_extent_km(sic, area)
        print(type(ext))
        # Store the computed extent along with the region name and year in the list
        

        # Clean up memory after each iteration to avoid excessive memory usage
        #del ds, sic, ext
        
    # # Convert the list of extents into a pandas DataFrame and export it to a CSV file
    # df = pd.DataFrame(extents)
    # df.to_csv(f'annualized_extent_{name}.csv', index=False)  # Save the results for each region

    # # Clean up memory after processing each region
    # del alaska_shp_proj


/Users/sunbak/miniforge3/envs/seaice/lib/python3.9/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'melt_onset_day_cdr_seaice_conc' has multiple fill values {-1, 255} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


Unable to load data: 'NoneType' object is not subscriptable


/Users/sunbak/miniforge3/envs/seaice/lib/python3.9/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'melt_onset_day_cdr_seaice_conc' has multiple fill values {-1, 255} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


<xarray.DataArray 'seaice_extent' (time: 365)> Size: 3kB
dask.array<stack, shape=(365,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 3kB 2022-09-01 2022-09-02 ... 2023-08-31
    spatial_ref  int64 8B 0


In [2]:
ds, area = sic_m.subset_dim([f'{lastyear-1}-09-01', f'{lastyear}-08-31'], alaska_shp_proj)

# Format the sea ice concentration data to binary (0 or 1) using a threshold of 0.15
sic = sic_m.format_sic(ds, 0.15)  # Sea ice concentration thresholding

# Compute the sea ice extent in square kilometers for the subset data
ext = sic_m.compute_extent_km(sic, area)


In [7]:
ext_df = (ext
        .to_dataframe()
        .reset_index()
        .drop(['spatial_ref'], axis='columns')
        .rename(columns={'time': 'date'})
        .set_index('date'))

In [8]:
ext_df


,time,seaice_extent
0,2022-09-01,57659.964622
1,2022-09-02,39863.199883
2,2022-09-03,53853.639689
3,2022-09-04,51328.828894
4,2022-09-05,57644.400417
...,...,...
360,2023-08-27,8029.624796
361,2023-08-28,8029.624796
362,2023-08-29,8029.624796
363,2023-08-30,8029.624796
